In [1]:

# %%
import pandas
import agentpy as ap
import seaborn as sns
import networkx as nx
import random as random
from math import sqrt 
from itertools import accumulate
from numpy.random import choice
import statistics
import math
from heapq import nlargest
import os
import segregation 
import geopandas as gpd
from libpysal.weights import Kernel
from shapely.geometry import Point
from libpysal import weights
import numpy as np
from scipy.integrate import quad
from segregation.multigroup import MultiDissim, MultiInfoTheory
import matplotlib.pyplot as plt
import pandas as pd

import csv
import regex
from pathlib import Path  
import pickle

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import geopandas as gpd
import splot
import libpysal


import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from esda.moran import Moran


# define Segregation Model

In [ ]:
### Define Agent Class: Person

class Person(ap.Agent):
    
    def setup(self):
        """ Initiate agent attributes. """
        
        # Segregation attributes 
        self.grid = self.model.grid
        self.group = None

        
        # attributes for reporters
        self.share_similar = 0
        self.share_ingroup = 0
        self.share_outgroup = 0
        
   
        # grid attributes
        self.coord = None
        
        # NW identifiers
        self.degree = None
        
        # decision threshold 
        self.happy = False

    
   
    def get_friends(self,G):
        
        connected_nodes = [to for (fr, to) in G.edges(self.node)]
        connected_agents = [self.model.node_to_agent[node] for node in connected_nodes]
        return connected_agents
    

    def find_new_friend_like_you(self, you):
        G=self.network_caveman.graph
        non_friends = self.get_non_friends(G=self.network_caveman.graph)
        non_friends_like_you = [agent for agent in non_friends if agent.group == you.group and agent != you]
        new_friend = random.choice(non_friends_like_you)
        G.add_edge(self.node, new_friend.node)
        return self, new_friend

    def get_non_friends(self,G):
        friends = self.get_friends(G=G)
        all_agents = [agent for agent in self.grid.agents]
        non_friends = [agent for agent in all_agents if not agent in friends]
        return non_friends
    
    def ditch_friend_like_you(self, you):
        G=self.network_caveman.graph
        friends = self.get_friends(G=self.network_caveman.graph)
        possible_friends_to_ditch = [agent for agent in friends if agent.group == you.group and agent != you]
        if len(possible_friends_to_ditch):
            ditch_friend = random.choice(possible_friends_to_ditch) 
            G.remove_edge(self.node, ditch_friend.node)
        else: 
            ditch_friend = random.choice(friends)

        return self, ditch_friend
     
    
    def setup_NW(self):
        """ This funciton is only called once in model setup.
        Steps:
        1.  Assign ties to agent (agent.kin), this is an iterator
        2.  Randomly choose a group (race/ethnicity) for one agent of the group, give all connected agents 
            within the clique the same group membership.
        3.  Randomly choose self.p.n_ties_outgroup naby ties of which to change group assignment. This allows
            for a deterministic way of defining homophily.
            
        
        """
        for agent in self.network_caveman.agents:
            agent.kin = agent.network_caveman.neighbors(agent)

            agent.random = self.model.random
            agent.group = self.random.choice(range(self.p.n_groups))

            for kin in agent.kin:
                kin.group=agent.group 
            
            ingroup_members = np.array([])
            
            for n in agent.network_caveman.neighbors(agent):
                if n.group == agent.group:
                    ingroup_members = np.append(ingroup_members, n)
            
            len_ingroup_members=len(ingroup_members)
            len_outgroup_members=self.p.size_cliques-1-len_ingroup_members
  

            while self.p.n_ties_outgroup >len_outgroup_members:

                reassign_group=self.random.choice(ingroup_members)
                if agent.group==1:
                    reassign_group.group=0                
                if agent.group==0:
                    reassign_group.group=1
                
                ingroup_members = np.array([])
                for n in agent.network_caveman.neighbors(agent):
                    if n.group == agent.group:
                        ingroup_members = np.append(ingroup_members, n)
                
                
                
                len_ingroup_members=len(ingroup_members)
                len_outgroup_members=self.p.size_cliques-1-len_ingroup_members

           
            # prepare degree variable
            self.degree=self.network_caveman.graph.degree[self.node]

                    
            self.org_connected = [to for (fr, to) in self.network_caveman.graph.edges(self.node)]
            
   
     
    def find_missing_group(self):
        """ For agents that have a missing degree, find which group is missing
        """
        #    connected to self.node and agent level
        connected_nodes = [to for (fr, to) in G.edges(self.node)]
        connected_agents = [self.model.node_to_agent[node] for node in connected_nodes]
        
        same_group = [agent for agent in connected_agents if agent.group == self.group]
        different_group = [agent for agent in connected_agents if agent.group != self.group]

    def find_available_agents(self, group, G):
        """
        Find a new agent as a function of their group
        
        group = agent.group 0 or 1
        """
        G=self.network_caveman.graph
        #    A, connected to self.node and agent level
        connected_nodes = [to for (fr, to) in G.edges(self.node)]
        connected_agents = [self.model.node_to_agent[node] for node in connected_nodes]
        
        #    B, particular group
        group_agents = [agent for agent in self.grid.agents if agent.group == group]
        
        #    Venn diagram 
        non_friend_group_agents = [agent for agent in group_agents if agent not in connected_agents]
        
        print(f'agents that have group x and are not yet connected{non_friend_group_agents}')

        
        return non_friend_group_agents

    def find_agents_missing_degree(self):
        """
        Find a new agent that has less than degree specified in parameters
        
        degree = model.p.size_cliques -1 
                """
        p_degree = self.model.p.size_cliques -1 
        
        print(f'size cliques is {self.model.p.size_cliques} and degree is {p_degree}')

        agents_missing = [agent for agent in self.grid.agents if agent.degree < p_degree]
        print(f'agents that dont have enough friends{agents_missing}')
        
        return agents_missing
    
    
    def tie_composition(self):
        """
        Find the number of ingroup and outgroup ties per agent
                """
        G=self.network_caveman.graph
        #    A, connected to self.node and agent level
        connected_nodes = [to for (fr, to) in G.edges(self.node)]
        connected_agents = [self.model.node_to_agent[node] for node in connected_nodes]
        
        #    B, particular group
        len_ingroup_ties = len([agent for agent in connected_agents if agent.group == self.group])
        len_outgroup_ties = len([agent for agent in connected_agents if agent.group != self.group])
        
        
        return len_ingroup_ties, len_outgroup_ties




    def drop_random_edges(self):
        """Steps:
        1. Access agent's node and all edges from said node
        2. Choose a random edge from node
        3. Remove self.model.p.n_edges_drop edges
        """
 
        G=self.network_caveman.graph
        print(f'node of {self} is {self.node}')

        for _ in range(self.model.p.n_edges_drop):
            
            agent_ties = G.edges(self.node)
            #print(f'nodes connected to {self.node} are {agent_ties}')

            edges = list(agent_ties) 
            #nonedges = list(nx.non_edges(G))
            #choose a random edge to remove
            if len(edges):
                chosen_edge = random.choice(edges)
            # delete chosen edge
                G.remove_edge(chosen_edge[0], chosen_edge[1])
             #   print(f'edge {chosen_edge} removed from the graph')

        

    def calculate_degree(self,G):
        # update degree variable
        #self.degree=self.network_caveman.graph.degree[self.node]
        self.degree=G.degree[self.node]
        #print('self.degree:  {}'.format(self.degree)) 
    

    def position(self):
        self.x = self.grid.positions[self][0]
        self.y = self.grid.positions[self][1]

        
    def update_happiness_network(self):
        # use distance to network as decision crit to decide if move or not
        # Set up segregation measurement
        neighbors = self.grid.neighbors(self)
        connected_agents=  [a for a in self.grid.agents if a in self.network_caveman.neighbors(self)] # agent level

        
        similar = len([n for n in neighbors if n.group == self.group])
        ln = len(neighbors)
        self.share_similar = similar / ln if ln > 0 else 0
        
        ties = len([n for n in neighbors if n in connected_agents])
        self.share_ties = ties / ln if ln > 0 else 0
        self.happy = self.share_ties >= self.p.want_ties # agent is happy if there are self.p.want_ties in the
	
    def find_new_home(self):
        """ Move to free spot. 
        Condition: there are more network ties around the new spot than threshold 
          
          ...and update free spots. """
        empty_spots=self.model.grid.empty
        connected_agents=  [a for a in self.grid.agents if a in self.network_caveman.neighbors(self)]
        
        list_iterator=list(range(len(empty_spots))) # create an iterator for all empty spots
        random.shuffle(list_iterator) # shuffle order in which to assess neighborhoods
        for i in list_iterator: # this ensures that the order of neighborhoods is pseudo random
            
            
            self.grid.move_to(self, empty_spots[i])
            neighbors_newspot = self.grid.neighbors(self)

            ln_newspot = len(neighbors_newspot)
            ties_newspot = len([n for n in neighbors_newspot if n in connected_agents])
            share_ties_newspot = ties_newspot / ln_newspot if ln_newspot > 0 else 0

            #print(f'share ties new spot is {share_ties_newspot} share ties old spot is  {self.share_ties} ')

            
            if share_ties_newspot>self.share_ties:
                #print(f'share ties new spot is {share_ties_newspot} which is  larger than {self.share_ties} ')
                i=100
            
            elif share_ties_newspot<self.share_ties:
                #print(f'share ties new spot is {share_ties_newspot} which is  smaller than {self.share_ties} ')
                pass
            
            elif share_ties_newspot==self.share_ties:
                #print(f'share ties new spot is {share_ties_newspot} which is equal to  {self.share_ties} ')
                pass
            
        self.model.unhappy = self.model.agents.select(self.model.agents.happy == False)


# ## Define Model class

# %%
class SegregationModel(ap.Model):

    
    def setup(self):  
        ####################################################################
        # Parameters
        s = self.p.size
        self.central_agents = False
        self.count=0
        self.rewired_steps=0
        
        self.p.n_cliques=int(self.p.n/self.p.size_cliques)
        print('parameters:  {}'.format(self.p))


        self.dt_steps = pd.DataFrame(columns=[ 't', 'N unhappy Agents' , 'segregation'  , 'moransI'])


        ####################################################################

     
        # caveman graph
        caveman = nx.caveman_graph(
            self.p.n_cliques, 
            self.p.size_cliques)
        
        
       
       ####################################################################
    
        # create grid
        self.grid = ap.Grid(self, (s, s), track_empty=True)

        # Create agents based on Person class
        self.agents = ap.AgentList(self, self.p.n, Person)
        

        # add agents to grid
        self.grid.add_agents(self.agents, random=True, empty=True) 


        ####################################################################\
        # introduce agent network
        if self.p.create_networks==True:

        
            ####################################################################\
            # networks
            # Setup fully connected caveman graph 
            
            self.network_caveman = self.agents.network_caveman = ap.Network(self, caveman)

            self.network_caveman.add_agents(self.agents, self.network_caveman.nodes)
            #print('nw agents= {}'.format(self.network_kin.nodes))   

            self.agents.node = self.network_caveman.nodes
            
            # create a dictionary for nodes - agents
            self.node_to_agent = dict(zip(self.agents.node, self.agents))
            
    
  



            self.agents.setup_NW()

        ####################################################################\


            self.agents.position()

        
            self.agents.calculate_degree(G=self.network_caveman.graph)


            # control agent poulation
            n_group1 = len(self.agents.select(self.agents.group == 1))
            n_group0 = len(self.agents.select(self.agents.group ==0))

            group_prop= n_group1/self.p.n
            print('group_prop:  {}'.format(group_prop))

            while group_prop!=0.5:
                self.agents.setup_NW()

                # control agent poulation
                n_group1 = len(self.agents.select(self.agents.group == 1))
                n_group0 = len(self.agents.select(self.agents.group ==0))

                group_prop= n_group1/self.p.n
                print('group_prop while loop:  {}'.format(group_prop))

            G=self.network_caveman.graph
            #nx.write_gexf( G , "test.edgelist" )




        
    
            #call rewire function
            rewired_steps = 0
            # calculate clustering
            clustering=nx.average_clustering(self.network_caveman.graph)
            # while clustering < clustering_threshold, run rewiring 
            while clustering > self.p.clustering_threshold:
                self.rewire_edges(self.p.steps_rewire)
                G = self.network_caveman.graph
                clustering=nx.average_clustering(self.network_caveman.graph)
                #print('average_clustering in loop:  {}'.format(nx.average_clustering(G)))
                #print(nx.community.modularity(G, nx.community.label_propagation_communities(G)))
                rewired_steps = rewired_steps +1

            # check for self loops
            self.network_caveman.graph.remove_edges_from(nx.selfloop_edges(self.network_caveman.graph))

            # Define viz parameters
            color_dict = {0:'Green', 1:'Grey'}
            colors = [color_dict[c] for c in self.agents.group]
        
            nx.draw(self.network_caveman.graph, node_size=30, node_color=colors)
            plt.show()


            G = self.network_caveman.graph
            print('average_clustering:  {}'.format(nx.average_clustering(G)))

            # Define viz parameters
            color_dict = {0:'Green', 1:'Grey'}
            colors = [color_dict[c] for c in self.agents.group]
            nx.draw(G, node_size=50, node_color=colors)
            plt.show()
            
            print(nx.community.modularity(G, nx.community.label_propagation_communities(G)))
            self.rewired_steps= rewired_steps


        
            self.agents.calculate_degree(G=self.network_caveman.graph)
        ####################################################################\


        ####################################################################\


        # introduce agent network
        else:
            if self.p.create_networks==False:
                print(self.p)

                # load egde list

                edgelist = pd.read_csv(f"1_createnetworks/network_files/edgelist_cliquesize{self.p.size_cliques}_cluster{self.p.clustering_threshold}_tiesoutgroup{self.p.n_ties_outgroup}.csv",on_bad_lines='skip')
                H = nx.from_pandas_edgelist(edgelist)    

       
                attributes_df = pd.read_csv(f'1_createnetworks/network_files/attributes_cliquesize{self.p.size_cliques}_cluster{self.p.clustering_threshold}_tiesoutgroup{self.p.n_ties_outgroup}.csv')
                attr_dict=attributes_df.set_index('node_label').to_dict(orient='index')


        
                ####################################################################\
                # networks        

                self.network_caveman = self.agents.network_caveman = ap.Network(self, H)
                self.network_caveman.add_agents(self.agents, self.network_caveman.nodes)
                self.agents.node = self.network_caveman.nodes


                self.node_to_agent = dict(zip(self.agents.node, self.agents))


                 # assign nodes the attributes from- dict
                _g = self.network_caveman.graph

                node_to_agent = {}
                agent_to_node = {}
                # Create 
                for agent_node, agent in zip(_g.nodes, self.agents):
                    #set attributes
                    for attr in attr_dict[agent_node.label].keys():
                        setattr(agent, attr, attr_dict[agent_node.label][attr])
            
                    
                    #Populate the dictionaries that gets the node from the agent and vice versa
                    agent_to_node[agent] = agent_node
                    node_to_agent[agent_node] = agent





                # Define viz parameters
                color_dict = {0:'Green', 1:'Grey'}
                colors = [color_dict[c] for c in self.agents.group]
        
                nx.draw(self.network_caveman.graph, node_size=30, node_color=colors)
                plt.show()



        

        



        # set up cavemangraph between neighbors
        
        # 1. select p.n/p.clique_size agents 
        # 2. start with one of the selected agents
        # 3. Identify the p.clique_size -1 closest agents that fullfill homophily criteria
        # 4. Add edges between all of them
        # 5. repeat for all agents selected in 3
  

    def rewire_edges(self, m):
        
        G = self.network_caveman.graph
        active_agent = random.choice(self.agents)
        friend = random.choice(active_agent.get_friends(G=self.network_caveman.graph))
        original_friend = friend
        G.remove_edge(active_agent.node, friend.node)
        for i in range(0,m):
            active_agent, new_friend  = active_agent.find_new_friend_like_you(friend)
            #Change focus to new agent
            friend = active_agent
            active_agent = new_friend

            active_agent, ditch_friend = active_agent.ditch_friend_like_you(friend)
            friend = active_agent
            active_agent = ditch_friend
 


    def update(self):

        self.agents.update_happiness_network()
        self.unhappy = self.agents.select(self.agents.happy == False)
        print('N unhappy:  {}'.format(len(self.unhappy)))
        self.record('N unhappy Agents', len(self.unhappy))


        self.record('segregation', round(sum(self.agents.share_similar) / self.p.n, 2))
        self.record('moransI', self.get_moransI())
        print('segregation:  {}'.format(round(sum(self.agents.share_similar) / self.p.n, 2)))

        # Create a list       
        new_row = [self.count, len(self.unhappy), self.get_segregation(),self.get_moransI() ]
        print(new_row)


        # Append the list as a new row to the DataFrame
        self.dt_steps.loc[len(self.dt_steps)] = new_row


        # Stop simulation if all are happy
        if len(self.unhappy) <= 4:
            self.stop()
        

    def step(self):        
        # run iterator of steps
        self.count = self.count + 1
    
        # Move unhappy people to new location
        self.unhappy.find_new_home()  
        print('self.count:  {}'.format(self.count))

    
    def gen_gdf(self):

        self.agents.position()

        # Assuming agents_list is your list of agents with x, y, and group attributes
        geometry = [Point(agent.x, agent.y) for agent in self.agents]
        gdf = gpd.GeoDataFrame(geometry=geometry, data={'group': [agent.group for agent in self.agents]})

        gdf = gdf.assign(
            group_0 = gdf['group'].map({0:1,1:0}),
            group_1 = gdf['group']
        )
        
        gdf = gdf.set_crs('epsg:3857')
        #print(gdf.crs is None)


        return(gdf)



    def get_segregation(self):
        # Calculate average percentage of similar neighbors
        return round(sum(self.agents.share_similar) / self.p.n, 2)
    
    def get_clustering(self):
        # Calculate average percentage of similar neighbors
        G = self.network_caveman.graph
        return nx.average_clustering(G)
    
    def get_ingroup_ties(self):
        # Calculate average percentage of in group ties 
        return round(sum(self.agents.share_ingroup)/ self.p.n,2)
    def get_outgroup_ties(self):
        # Calculate average percentage of out group ties 
        return round(sum(self.agents.share_outgroup)/ self.p.n,2)


    def get_homophily(self):
        return round(sum(self.agents.homophily/ self.p.n,2))
    
    def get_modularity(self):
        G = self.network_caveman.graph
        #print(nx.community.modularity(G, nx.community.label_propagation_communities(G)))

       # communities = list(nx.community.girvan_newman(G))
        return nx.community.modularity(G, nx.community.label_propagation_communities(G))



    def seg_curve(self):
        gdf=self.gen_gdf()

        # Plot segregation curves over parameter r
        r_values = list(range(1, self.p.distance + 1))
        segregation_values = []

        for r in r_values:
            spatial_multi_info= MultiInfoTheory(gdf, groups=['group_0','group_1'], distance=r)
            segregation_values.append(spatial_multi_info.statistic)

        plt.plot(r_values, segregation_values, marker='o')
        plt.xlabel('Distance')
        plt.ylabel('Segregation (Information Theory Index)')
        plt.title('Segregation Profiles')
        plt.show()

        df = pd.DataFrame(segregation_values)

        return segregation_values
    

    def get_moransI(self):
        gdf=self.gen_gdf()

        y = gdf['group'].values
        w = Queen.from_dataframe(gdf, use_index=False)
        moran = Moran(y, w)
        print(f'morans I: {moran.I}')

        return moran.I
    



    

    

    def end(self): 

        self.dt_steps.to_csv(f'test1220.csv', index=False) 
        lst = os.listdir(f'3_simulation/t_seg/cliquesize{self.p.size_cliques}_cluster{self.p.clustering_threshold}_tiesoutgroup{self.p.n_ties_outgroup}/') # directory path
        number_files = len(lst)
        #self.dt_steps.to_csv(f'3_simulation/t_seg/cliquesize{self.p.size_cliques}_cluster{self.p.clustering_threshold}_tiesoutgroup{self.p.n_ties_outgroup}/reporterst_sampleid_{number_files}.csv', index=True)

         
        # Measure reporters at the end of the simulation
        self.report('segregation', self.get_segregation())
        self.report('Morans I', self.get_moransI())
        self.report('N unhappy', len(self.unhappy))
        self.report('Steps', self.count)
        self.report('clustering', self.get_clustering())
        self.report('modularity', self.get_modularity())
        self.report('seg_curve', self.seg_curve())
        self.report('distance_values', list(range(1, self.p.distance + 1)))
        


       


        









# define parameters

In [ ]:



parameters = {
    'n_groups': 2, # Number of groups (ethnic)
    'size': 30, # Height and length of the grid
    'steps': 50000,  # Maximum number of steps
    'size_cliques':20 , # # how large are cliques (size_cliques - 1 = degree)
    'steps_rewire': 1,  # How many iterations of rewiring
    'n':800, # population size
    'want_ties':0.3, # preference threshold
    'n_ties_outgroup':0,  # defines homophily
    'clustering_threshold': 0.4, # network clustering
    'distance':20, # calculate H index over distance
    'create_networks':False # if True: create networks, if False: network attributes and edge lists are loaded

}

# single run 
## create animated grid

In [ ]:
def animation_plot(model, ax):
    # Visualize the group attribute of the grid
    group_grid = model.grid.attr_grid('group')
    ap.gridplot(group_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Segregation model \n Time-step: {model.t}")

# Create a figure for the animation
fig, ax = plt.subplots()

# Initialize the model with your parameters
model = SegregationModel(parameters)

# Define the step and skip parameters
# `step` defines the number of steps in the animation
# `skip` determines the interval of steps to skip during the animation
# use for model iterations with high required step count. 

#step = 50000  # Total number of animation steps
#skip = 49000   # Skip the first steps to display

# Animate the model
animation = ap.animate(model, fig, ax, animation_plot, steps=step, skip=skip)

# Display the animation in a Jupyter notebook
IPython.display.HTML(animation.to_jshtml())


# run experiments
## c = 400

In [ ]:
model = SegregationModel(parameters)

parameters_multi = dict(parameters) 


parameters_multi.update({
    'size_cliques':400,
    'n_ties_outgroup':ap.Values(0)
})


sample = ap.Sample(parameters_multi, n=5)
exp = ap.Experiment(SegregationModel, sample, iterations=5)
results_400= exp.run()
results_400.save()

## c = 200

In [ ]:
model = SegregationModel(parameters)

parameters_multi = dict(parameters) 


parameters_multi.update({
    'size_cliques':200,
    'n_ties_outgroup':ap.Values(0, 40, 100),
    'clustering_threshold':ap.Values(1, 0.8,0.6,0.4)
})


sample = ap.Sample(parameters_multi, n=5)
exp = ap.Experiment(SegregationModel, sample, iterations=5)
results_200= exp.run()
results_200.save()

## c= 100

In [ ]:
model = SegregationModel(parameters)

parameters_multi = dict(parameters) 


parameters_multi.update({
    'size_cliques':100,
    'n_ties_outgroup':ap.Values(0, 20, 50),
    'clustering_threshold':ap.Values(1, 0.8,0.6,0.4)
})


sample = ap.Sample(parameters_multi, n=5)
exp = ap.Experiment(SegregationModel, sample, iterations=5)
results_100= exp.run()
results_100.save()

## c= 20

In [ ]:
model = SegregationModel(parameters)

parameters_multi = dict(parameters) 


parameters_multi.update({
    'size_cliques':20,
    'n_ties_outgroup':ap.Values(0, 4, 10),
    'clustering_threshold':ap.Values(1, 0.8,0.6,0.4)
})


sample = ap.Sample(parameters_multi, n=5)
exp = ap.Experiment(SegregationModel, sample, iterations=5)
results_20= exp.run()
results_20.save()

## c= 5

In [ ]:
model = SegregationModel(parameters)

parameters_multi = dict(parameters) 


parameters_multi.update({
    'size_cliques':5,
    'n_ties_outgroup':ap.Values(0, 1, 2),
    'clustering_threshold':ap.Values(1, 0.8,0.6,0.4)
})


sample = ap.Sample(parameters_multi, n=5)
exp = ap.Experiment(SegregationModel, sample, iterations=5)
results_5= exp.run()
results_5.save()